<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Deep Learning for NLP
  </div> 
  
<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Part I - 3 <br><br><br>
  Language Modeling
  </div> 

  <div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 20px; 
      text-align: center; 
      padding: 15px;">
  </div> 

  <div style=" float:right; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  Jean-baptiste AUJOGUE
  </div> 

### Part I

1. Word Embedding

2. Sentence Classification

    _Applications :_
    
    - Extractive Summarization
    - Sentiment Analysis
    - Text segmentation


3. <font color=red>**Language Modeling**</font>

4. Sentence tagging

    _Applications :_
    
    - Part-of-speech Tagging
    - Named Entity Recognition
    - Automatic Value Extraction
    


### Part II

5. Auto-Encoding

6. Machine Translation

7. Text Classification




### Part III

8. Abstractive Summarization

9. Question Answering

10. Chatbot


</div>

***

In [1]:
# path to anaconda : C:\ProgramData\Anaconda3

from __future__ import unicode_literals, print_function, division
import os
from io import open
import unicodedata
import string
import time
import math
import re
import json
import random
import itertools
import copy

In [2]:
import sys
sys.version

'3.7.0 (default, Jun 28 2018, 08:04:48) [MSC v.1912 64 bit (AMD64)]'

In [3]:
# for manipulating dataframes, arrays, containers...
import numpy as np
#np.set_printoptions(threshold=np.nan)
import pandas as pd
import unidecode
from unidecode import unidecode

# equivalent of numpy for very large arrays (such as a database of pretrained word vectors)
# optimized for SQL-like operations, not for math computations
# see https://bcolz.readthedocs.io/en/latest/intro.html

# installer avec la commande : conda install bcolz
import bcolz

import pickle

In [4]:
# for deep learning
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

print(torch.__version__)

use_cuda = False
print(torch.cuda.is_available())

import warnings
warnings.filterwarnings("ignore")

0.4.1
True


In [5]:
# for NLP
import gensim
from gensim.models import KeyedVectors

import nltk
#nltk.download()
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem.porter import PorterStemmer

#from textblob import TextBlob

In [6]:
import warnings
warnings.filterwarnings('ignore')

USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)
device = torch.device("cuda" if USE_CUDA else "cpu")

True


In [9]:
path_to_rep = 'C:\\Users\\jbaujogue\\Desktop\\NLP\\démonstrateurs NLP'
path_to_NLP = 'C:\\Users\\jbaujogue\\Desktop\\NLP'

<a id="corpus"></a>

# 0 Préparation

<a id="import"></a>

### 0.1 Import du corpus

[Retour à la table des matières](#plan)

Import du corpus de tickets Lilly, avec chaque ticket sous la forme d'une liste nettoyée de tokens

In [8]:
xl = pd.ExcelFile(path_to_rep + '\data\\tickets_Lilly_05_03_2019_attributs_additionnels.xlsx' )
df_total = xl.parse('Sheet1')

df_Lilly = df_total.dropna(subset = ['Short Description', 
                                     'Description', 
                                     'Analyse'])

print(df_Lilly.shape)
df_Lilly.head()

(3394, 12)


,Number,Created,Problem,Short Description,Description,Resolution Code,Resolve Notes,Résumé,Résolution,Catégorie,QAimpact,Analyse
0,INC2976351,2016-06-01 06:54:32,NaN,AST : PMX-PKG : Démarrage étape de production ...,Impossible de démarrer l'étape de production s...,People/System use,• [Résumé] Impossible de démarrer l'étape de p...,Impossible de démarrer l'étape de production s...,Fermeture de la session concernée via la Conso...,HE/PKG/Limos,Sans impact,Hscope/Duplicate/Cslt/Action
1,INC2976936,2016-06-01 09:56:53,NaN,PMX-PKG : Alarme type 2 non vue par l'équipement,Une alarme s'est déclenchée sur le limos LPF-E...,NaN,• [Résumé]\n\n• [Catégorie] \n\n• [Résolution]...,NaN,0,NaN,Sans impact,Matrikon/Alarme
2,INC2977191,2016-06-01 10:51:31,NaN,PMX-PKG : ticket journalier MO3524 du 01/06/2016,01/06/2016 00:00:00 => 01/06/2016 10:47:14\n-...,Monitoring,• [Catégorie] Monitoring des erreurs d'interfa...,NaN,NaN,Interface,Sans impact,Dailyinterfacemonitoring
3,INC2981590,2016-06-02 09:55:10,NaN,PMX-PKG : ref_relation = WESSTOCKREMOVAL (LC ...,Message envoyé le 01/06/2016 12:57:07\n\nid = ...,NaN,• [Résumé] Message d'erreur reçu sur l'interfa...,Message d'erreur reçu sur l'interface WES,Régulation effectuée par Stephane Seyller du b...,HE/WES,Sans impact,Lcmovementerror
4,INC2981597,2016-06-02 09:56:36,NaN,PMX-PKG : ref_relation = WESSTOCKREMOVAL (LC =...,Message envoyé le 01/06/2016 12:59:07\n\nid = ...,NaN,• [Résumé] Message d'erreur reçu sur l'interfa...,Message d'erreur reçu sur l'interface WES,Régulation effectuée par Stephane Seyller du b...,HE/WES,Sans impact,Lcmovementerror


In [20]:
def fusion(liste1, liste2):
    return [a.lower() + ' ' + b.lower() for a,b in zip(liste2, liste1)]

In [59]:
titres = ['Short Description : ', 'Description : ', 'Resolve Notes : ']
X = [' . '.join(fusion([str(el) for el in els], titres )) \
            for els in zip(df_Lilly['Short Description'].values.tolist(),
                           df_Lilly['Description'].values.tolist(),
                           df_Lilly['Resolve Notes'].values.tolist()
                          )]
X = [[[word.strip() for word in sentence.split()] + ['.'] for sentence in text.split('.')] for text in X]

Formation du language

In [60]:
class Lang:
    def __init__(self, name, init = 3):
        self.name = name
        if init == 0 :
            self.word2index = {}
            self.word2count = {}
            self.index2word = {}
            self.n_words = 0
        elif init == 3 :
            self.word2index = {"SOS": 0, "EOS": 1, "UNK": 2}
            self.word2count = {"SOS": 0, "EOS": 0, "UNK": 0}
            self.index2word = {0: "SOS", 1: "EOS", 2: "UNK"}
            self.n_words = 3

        
    def addWord(self, word):
        '''Add a word to the language'''
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            
            
    def addSentence(self, sentence):
        '''Add to the language all words of a sentence'''
        if type(sentence) == str :
            for word in nltk.word_tokenize(sentence): # sentence.split()
                self.addWord(word)
        elif type(sentence) == list :
            for word in sentence:
                self.addWord(word)            
            
            
    def addDescriptions(self, descriptions, lvl = 1):
        '''Add to the language all words contained into : either all user utterances 
          (if i = 0) or all bot utterances (if i = 1), of a list of dialogues'''
        for description in descriptions :
            
            # si la description est une seule ligne
            if type(description) == str :
                    try :
                        if lvl == 2 :
                            for sentence in nltk.sent_tokenize(description) :
                                self.addSentence(sentence)
                        else :
                            self.addSentence(description)
                    except IndexError:
                        print("Problem with {}".format(description))
                        
            # si la description est une liste de lignes
            elif type(description) == list :
                for line in description:
                    try :
                        self.addSentence(line)
                    except IndexError:
                        print("Problem with {}".format(line))
                              
            # sinon
            else :
                print("Problem with {}".format(description))

In [167]:
def generateLanguageFromWordList(liste, init = 0):
    lang = Lang('descriptions', init)
    lang.addSentence(liste)
    print("Mots comptés : ", lang.n_words)
    return lang


def generateLanguageFromNumpy(descriptions, init = 0, lvl = 1):
    lang = Lang('descriptions', init)
    lang.addDescriptions(descriptions)
    print("Mots comptés : ", lang.n_words)
    return lang


def generateLanguageFromPanda(df, i):
    descriptions = df.ix[:, i].values
    lang = Lang('descriptions')
    lang.addDescriptions(descriptions)
    print("Mots comptés : ", lang.n_words)
    return lang

In [61]:
lang = generateLanguageFromNumpy(X, init = 3, lvl = 2)

Mots comptés :  6474


<a id="embedding"></a>

### 0.2 Word Embedding

[Retour à la table des matières](#plan)

#### 0.2.1 Modèle Skip-gram de Gensim

In [66]:
from gensim.models import Word2Vec
import multiprocessing

In [67]:
data_gensim_skipgram = flatten(X) # ------ [[str]] : al list of list of tockens

In [68]:
skipgram_gensim = Word2Vec(data_gensim_skipgram, 
                           size = 100, 
                           window = 5, 
                           min_count = 1, 
                           negative = 20, 
                           iter = 500,
                           sg = 1,
                           workers = multiprocessing.cpu_count())

Obtenir la table des vecteurs

In [70]:
T = skipgram_gensim.wv.vectors
print('T est de type {} et de taille {}'.format(type(T), T.shape))

T est de type <class 'numpy.ndarray'> et de taille (6488, 100)


#### 0.2.2 Modèle Skip-gram sur mesure


In [54]:
def flatten(description) :
    '''Baisse le nombre de niveaux de 1 dans la description'''
    flatten = []
    for line in description :
        flatten += line + ['.']
    return flatten


def generateNgrams(descriptions, context_size = 5) :
    '''descriptions = [[str]]'''
    data = []
    for description in descriptions :
        line = flatten(description)
        line = ['SOS' for i in range(context_size)] + line + ['EOS' for i in range(context_size)] 
        for i in range(context_size, len(line) - context_size):
            context = line[i-context_size : i] + line[i+1 : i+context_size+1]
            target = [line[i]]
            data.append((context, target))
    return data

In [26]:
class SkipGram(nn.Module):
    def __init__(self, device, 
                 lang, 
                 context_size, 
                 embedding_dim, 
                 hidden_dim):
        
        super(SkipGram, self).__init__()
        self.device = device
        self.lang = lang
        self.output_lang = copy.copy(lang)
        self.output_lang.addDescriptions(['SOS', 'UNK', 'EOS'])
        self.context_size = 2 * context_size
        self.embedding = nn.Embedding(lang.n_words, embedding_dim)
        self.linear_1 = nn.Linear(embedding_dim, 2*context_size * hidden_dim)
        self.linear_2 = nn.Linear(hidden_dim, self.output_lang.n_words)
        
        
    def variableFromSentence(self, sentence):
        '''Turn a sentence into a torch variable, containing a list of indices according
           to a given language.'''
        indexes = [self.lang.word2index[word] for word in sentence if word in self.lang.word2index]          
        result = Variable(torch.LongTensor(indexes).view(1, -1)).to(self.device)
        return result
    

    def forward(self, word):
        input = self.variableFromSentence(word)                   # size = (1, 1)
        embed = self.embedding(input)#.view((1, -1))              # size = (1, embedding_dim)
        out = self.linear_1(embed).view((self.context_size, -1))  # size = (2 * context_size , hidden_dim)
        out = F.relu(out)                                         # size = (2 * context_size , hidden_dim)
        out = self.linear_2(out)                                  # size = (2 * context_size , lang.n_words)
        log_probs = F.log_softmax(out)                            # size = (2 * context_size , lang.n_words)
        return log_probs

In [27]:
class SkipGramTrainer(object):
    def __init__(self, 
                 device,
                 criterion = nn.NLLLoss(reduce = False), #nn.BCEWithLogitsLoss(), #nn.BCELoss(), 
                 optimizer = optim.SGD,
                 print_every=100):
        # relevant quantities
        self.device = device
        self.criterion = criterion
        self.optimizer = optimizer
        self.print_every = print_every
        
        
    def asMinutes(self, s):
        m = math.floor(s / 60)
        s -= m * 60
        return '%dm %ds' % (m, s)
    
    
    def timeSince(self, since, percent):
        now = time.time()
        s = now - since
        es = s / (percent)
        rs = es - s
        return '%s (- %s)' % (self.asMinutes(s), self.asMinutes(rs))
        
        
    def trainLoop(self, agent, context, word, weights, optimizer, learning_rate):
        """Performs a training loop, with forward pass and backward pass for gradient optimisation."""
        optimizer.zero_grad()
        agent.zero_grad()
        
        log_probs = agent(word)
        context_var = agent.variableFromSentence(context).view(-1)
        loss = self.criterion(log_probs, context_var)
        loss = torch.sum(weights * loss)
        loss_diff = 0
        for i in range(agent.context_size) :
            topv, topi = log_probs[i].data.topk(1)
            ni = topi[0][0]
            loss_diff = loss_diff + 1 if ni != context_var[i].data[0] else loss_diff
        loss.backward()
        optimizer.step()                                
        return loss.data[0], loss_diff
        
        
    def train(self,
              agent, 
              ngrams, 
              weights,
              n_iters = 100,
              n_epochs = None,
              learning_rate=0.01,
              random_state = 42
             ):
        """Performs training over a given dataset and along a specified amount of loops."""
        np.random.seed(random_state)
        start = time.time()
        optimizer = self.optimizer([param for param in agent.parameters() if param.requires_grad == True], lr=learning_rate)
        weights = Variable(torch.FloatTensor(weights)).to(self.device)
        print_loss_total = 0  
        print_loss_diff_mots_total = 0
        if n_epochs is None :
            for iter in range(1, n_iters + 1):
                couple = random.choice(ngrams)
                context = couple[0] 
                target = couple[1] 
                loss, loss_diff_mots = self.trainLoop(agent, context, target, weights, optimizer, learning_rate)
                # quantité d'erreurs sur la réponse i
                print_loss_total += loss
                print_loss_diff_mots_total += loss_diff_mots       
                if iter % (self.print_every) == 0:
                    print_loss_avg = print_loss_total / self.print_every
                    print_loss_diff_mots_avg = print_loss_diff_mots_total / self.print_every
                    print_loss_total = 0
                    print_loss_diff_mots_total = 0
                    print('%s (%d %d%%) %.4f %.2f' % (self.timeSince(start, iter / n_iters),
                                                 iter, iter / n_iters * 100, 
                                                      print_loss_avg, print_loss_diff_mots_avg))
        else :
            for epoch in range(n_epochs):
                print('epoch ' + str(epoch))
                np.random.shuffle(ngrams)
                for couple in ngrams :
                    context = couple[0] 
                    target = couple[1] 
                    loss, loss_diff_mots = self.trainLoop(agent, context, target, weights, optimizer, learning_rate)

                    # quantité d'erreurs sur la réponse i
                    print_loss_total += loss
                    print_loss_diff_mots_total += loss_diff_mots       

                    if iter % (self.print_every) == 0:
                        print_loss_avg = print_loss_total / self.print_every
                        print_loss_diff_mots_avg = print_loss_diff_mots_total / self.print_every
                        print_loss_total = 0
                        print_loss_diff_mots_total = 0
                        print('%s (%d %d%%) %.4f %.2f' % (self.timeSince(start, iter / n_iters),
                                                     iter, iter / n_iters * 100, 
                                                          print_loss_avg, print_loss_diff_mots_avg))

In [28]:
def weightsList(n):
    half = (n-1)/2
    weights = [n**2 -(half-i)**2 for i in range(n)]
    weights = [el/sum(weights) for el in weights]
    return weights

Entrainement du modèle

In [64]:
skipgram = SkipGram(device, lang, context_size = 5, embedding_dim = 75, hidden_dim = 75)
skipgram = skipgram.to(device)
skipgram_trainer = SkipGramTrainer(device = device)

In [65]:
Ngrams = generateNgrams(X, context_size = 5)

In [66]:
skipgram_trainer.train(skipgram, Ngrams, weights = weightsList(10), n_iters = 40000, learning_rate=0.01)
#skipgram_trainer.train(skipgram, Ngrams, weights = weightsList(10), n_iters = 30000, learning_rate=0.005)
#skipgram_trainer.train(skipgram, Ngrams, weights = weightsList(10), n_iters = 30000, learning_rate=0.0025)
#skipgram_trainer.train(skipgram, Ngrams, weights = weightsList(10), n_iters = 30000, learning_rate=0.001)

0m 1s (- 8m 21s) (100 0%) 8.6615 9.31
0m 2s (- 8m 14s) (200 0%) 8.3966 7.81
0m 3s (- 8m 14s) (300 0%) 7.9387 7.37
0m 4s (- 8m 11s) (400 1%) 7.6289 7.46
0m 6s (- 8m 8s) (500 1%) 7.3045 7.53
0m 7s (- 8m 5s) (600 1%) 7.2159 7.55
0m 8s (- 8m 5s) (700 1%) 6.9773 7.34
0m 9s (- 8m 4s) (800 2%) 7.3433 7.89
0m 11s (- 8m 3s) (900 2%) 6.9874 7.49
0m 12s (- 8m 0s) (1000 2%) 7.0193 7.57
0m 13s (- 7m 59s) (1100 2%) 6.9350 7.46
0m 14s (- 7m 58s) (1200 3%) 7.2537 7.85
0m 16s (- 7m 56s) (1300 3%) 6.8125 7.40
0m 17s (- 7m 55s) (1400 3%) 6.9897 7.67
0m 18s (- 7m 54s) (1500 3%) 6.8655 7.45
0m 19s (- 7m 54s) (1600 4%) 6.7800 7.33
0m 21s (- 7m 53s) (1700 4%) 6.8878 7.56
0m 22s (- 7m 51s) (1800 4%) 6.7829 7.45
0m 23s (- 7m 51s) (1900 4%) 6.7094 7.43
0m 24s (- 7m 49s) (2000 5%) 6.9370 7.81
0m 25s (- 7m 48s) (2100 5%) 6.8034 7.65
0m 27s (- 7m 46s) (2200 5%) 6.8269 7.61
0m 28s (- 7m 45s) (2300 5%) 6.7266 7.65
0m 29s (- 7m 44s) (2400 6%) 6.6305 7.67
0m 30s (- 7m 44s) (2500 6%) 6.6411 7.53
0m 32s (- 7m 44s) (2600

4m 13s (- 4m 8s) (20200 50%) 4.6532 6.74
4m 14s (- 4m 7s) (20300 50%) 4.5873 6.68
4m 15s (- 4m 5s) (20400 51%) 4.5567 6.74
4m 17s (- 4m 4s) (20500 51%) 4.6774 6.89
4m 18s (- 4m 3s) (20600 51%) 4.9188 7.06
4m 19s (- 4m 2s) (20700 51%) 4.5559 6.77
4m 20s (- 4m 0s) (20800 52%) 4.2201 6.46
4m 22s (- 3m 59s) (20900 52%) 4.3170 6.56
4m 23s (- 3m 58s) (21000 52%) 4.6413 6.99
4m 24s (- 3m 56s) (21100 52%) 4.5372 6.83
4m 25s (- 3m 55s) (21200 53%) 4.3307 6.49
4m 27s (- 3m 54s) (21300 53%) 4.6840 6.79
4m 28s (- 3m 53s) (21400 53%) 4.6583 6.91
4m 29s (- 3m 51s) (21500 53%) 4.3774 6.50
4m 30s (- 3m 50s) (21600 54%) 4.4222 6.63
4m 31s (- 3m 49s) (21700 54%) 4.6390 6.85
4m 33s (- 3m 48s) (21800 54%) 4.5037 6.67
4m 34s (- 3m 46s) (21900 54%) 4.6484 6.85
4m 35s (- 3m 45s) (22000 55%) 4.7608 7.11
4m 36s (- 3m 44s) (22100 55%) 4.3271 6.68
4m 38s (- 3m 43s) (22200 55%) 4.2208 6.14
4m 39s (- 3m 41s) (22300 55%) 4.3677 6.53
4m 40s (- 3m 40s) (22400 56%) 4.4364 6.70
4m 41s (- 3m 39s) (22500 56%) 4.3389 6.66

8m 18s (- 0m 1s) (39900 99%) 3.9669 6.40
8m 19s (- 0m 0s) (40000 100%) 4.2651 6.35


In [67]:
# sauver
#torch.save(skipgram.state_dict(), path_to_rep + '\saves\\Lilly_pkg_modele_linguistique_skipgram.pth')

# charger
#skipgram.load_state_dict(torch.load(path_to_rep + '\saves\\Lilly_pkg_modele_linguistique_skipgram.pth'))

Obtenir la table des vecteurs

In [68]:
T = skipgram.embedding.weight.cpu().detach().numpy()
print('T est de type {} et de taille {}'.format(type(T), T.shape))

T est de type <class 'numpy.ndarray'> et de taille (6474, 75)


L'alignement mot-indice est donné par le dictionnaire lang.word2index du language chargé dans le modèle

#### 0.2.3 Modèle Skip-gram de FastText

In [150]:
from gensim.models.fasttext import FastText as FT_gensim
from gensim.test.utils import datapath

In [191]:
fastText_gensim = FT_gensim(size = 100, 
                           window = 5, 
                           min_count = 1, 
                           negative = 20,
                           sg = 1)

In [153]:
def flatten(description) :
    '''Baisse le nombre de niveaux de 1 dans la description'''
    flatten = []
    for line in description :
        flatten += line
    return flatten

In [189]:
X_flat = [['SOS'] + flatten(text) + ['EOS'] for text in X]

In [192]:
fastText_gensim.build_vocab(sentences = X_flat)

In [193]:
fastText_gensim.train(sentences = X_flat, 
                      epochs = 500,
                      total_examples=fastText_gensim.corpus_count)

In [196]:
#fastText_gensim.save('Lilly_pkg_modele_linguistique_fastText')
#fastText_gensim = FT_gensim.load('Lilly_pkg_modele_linguistique_fastText')

In [194]:
print(fastText_gensim)

T = fastText_gensim.wv.vectors
print('T est de type {} et de taille {}'.format(type(T), T.shape))

FastText(vocab=6473, size=100, alpha=0.025)
T est de type <class 'numpy.ndarray'> et de taille (6473, 100)


In [195]:
lang = generateLanguageFromWordList(list(fastText_gensim.wv.vocab.keys()), init = 0)

Mots comptés :  6473


<a id="model_linguistique"></a>

# 1 Modèle Linguistique

Exemples d'implémentation en PyTorch :

- https://github.com/pytorch/examples/blob/master/word_language_model/model.py


Différentes architectures sont décrites dans la litérature :

- Regularizing and Optimizing LSTM Language Models - https://arxiv.org/pdf/1708.02182.pdf

Un modèle linguistique est intérressant en soi, mais peut aussi servir pour le pré-entrainement de couches basses d'un modèle plus complexe :

- Deep contextualized word representations - https://arxiv.org/pdf/1802.05365.pdf
- Improving Language Understanding by Generative Pre-Training - https://www.cs.ubc.ca/~amuham01/LING530/papers/radford2018improving.pdf
- Language Models are Unsupervised Multitask Learners - https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf

### 1.1 Le modèle linguistique

[Retour à la table des matières](#plan)

In [333]:
class LanguageModel(nn.Module):
    def __init__(self, device, 
                 lang,
                 embedding_weights, 
                 hidden_dim,
                 n_layers = 1, 
                 dropout = 0): 
        
        super(LanguageModel, self).__init__()
        # relevant quantities
        self.device = device
        self.lang = lang
        self.hidden_dim = hidden_dim
        self.dropout_p = dropout
        self.n_layers = n_layers               # number of stacked GRU layers
        self.output_dim = hidden_dim           # dimension of outputed rep. of words and utterance
        # parameters
        self.embedding = nn.Embedding(embedding_weights[0], embedding_weights[1]) if type(embedding_weights) == tuple else \
                         nn.Embedding.from_pretrained(torch.FloatTensor(embedding_weights), freeze=True)
        for p in self.embedding.parameters() :
            embedding_dim = p.data.size(1)
        self.dropout = nn.Dropout(p = dropout)
        self.gru = nn.GRU(embedding_dim, 
                          self.hidden_dim, 
                          n_layers,
                          dropout=(0 if n_layers == 1 else dropout), 
                          bidirectional = False)
        self.out = nn.Linear(self.hidden_dim, lang.n_words)
        #self.out.weight = self.embedding.weight
        
        
    # ------------- technical methods ------------------    
    def nbParametres(self) :
        count = 0
        for p in self.parameters():
            if p.requires_grad == True :
                count += p.data.nelement()
        return count
    

    def variableFromSentence(self, sentence):
        '''Turn a sentence into a torch variable, containing a list of indices according
           to a given language.'''
        indexes = [self.lang.word2index[word] for word in sentence if word in self.lang.word2index]          
        result = Variable(torch.LongTensor(indexes).view(-1, 1)).to(self.device)
        return result
    

    def initHidden(self): 
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_dim)).to(self.device)
    
    
    # ------------- main methods ------------------
    def generateWord(self, utterance, hidden = None):
        embeddings = self.embedding(utterance)                          # dim = (input_length, 1, embedding_dim)
        embeddings = self.dropout(embeddings)                           # dim = (input_length, 1, embedding_dim)
        hidden = self.initHidden() if hidden is None else hidden
        _, hidden = self.gru(embeddings, hidden)                        # dim = (input_length, 1, hidden_dim)
        hidden = self.dropout(hidden)
        log_probs = F.log_softmax(self.out(hidden[-1]))         # dim = (1, lang_size)
        return log_probs, hidden  

    
    def forward(self, sentence = ['SOS'], hidden = None, limit = 10, retain = False):
        result = sentence + ['\033[94m']
        count = 0
        stop = False
        utterance = self.variableFromSentence(sentence)
        while not stop :
            log_probs, hidden = self.generateWord(utterance, hidden)
            topv, topi = log_probs[0].data.topk(1)
            ni = topi.item()
            utterance = Variable(torch.LongTensor([[ni]])).to(self.device)
            result.append(self.lang.index2word[ni])
            count += 1
            if count == limit or (type(limit) == str and ni == self.lang.word2index[limit]) or count == 50 :
                stop = True
        if not retain :
            print(' '.join(result + ['\033[0m']))
            return
        else :
            return result, hidden   

In [390]:
class LanguageModel(nn.Module):
    def __init__(self, device, 
                 lang,
                 embedding_weights, 
                 hidden_dim,
                 n_layers = 1, 
                 dropout = 0): 
        
        super(LanguageModel, self).__init__()
        # relevant quantities
        self.device = device
        self.lang = lang
        self.hidden_dim = hidden_dim
        self.dropout_p = dropout
        self.n_layers = n_layers               # number of stacked GRU layers
        self.output_dim = hidden_dim           # dimension of outputed rep. of words and utterance
        # parameters
        self.embedding = nn.Embedding(embedding_weights[0], embedding_weights[1]) if type(embedding_weights) == tuple else \
                         nn.Embedding.from_pretrained(torch.FloatTensor(embedding_weights), freeze=True)
        for p in self.embedding.parameters() :
            embedding_dim = p.data.size(1)
        self.dropout = nn.Dropout(p = dropout)
        self.gru = nn.GRU(embedding_dim, 
                          self.hidden_dim, 
                          n_layers,
                          dropout=(0 if n_layers == 1 else dropout), 
                          bidirectional = False)
        self.out = nn.Linear(self.hidden_dim, lang.n_words)
        #self.out.weight = self.embedding.weight
        
        
    # ------------- technical methods ------------------    
    def nbParametres(self) :
        count = 0
        for p in self.parameters():
            if p.requires_grad == True :
                count += p.data.nelement()
        return count
    

    def variableFromSentence(self, sentence):
        '''Turn a sentence into a torch variable, containing a list of indices according
           to a given language.'''
        indexes = [self.lang.word2index[word] for word in sentence if word in self.lang.word2index]          
        result = Variable(torch.LongTensor(indexes).view(-1, 1)).to(self.device)
        return result
    

    def initHidden(self): 
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_dim)).to(self.device)
    
    
    # ------------- main methods ------------------
    def generateWord(self, utterance, hidden = None):
        embeddings = self.embedding(utterance)                          # dim = (input_length, 1, embedding_dim)
        embeddings = self.dropout(embeddings)                           # dim = (input_length, 1, embedding_dim)
        hidden = self.initHidden() if hidden is None else hidden
        _, hidden = self.gru(embeddings, hidden)                        # dim = (input_length, 1, hidden_dim)
        hidden = self.dropout(hidden)
        log_probs = F.log_softmax(self.out(hidden[-1]))         # dim = (1, lang_size)
        return log_probs, hidden  

    
    def forward(self, sentence = ['SOS'], hidden = None, limit = 10, retain = False, color_code = '\033[40m'):
        result = sentence + [color_code]
        count = 0
        stop = False
        utterance = self.variableFromSentence(sentence)
        while not stop :
            log_probs, hidden = self.generateWord(utterance, hidden)
            topv, topi = log_probs[0].data.topk(1)
            ni = topi.item()
            utterance = Variable(torch.LongTensor([[ni]])).to(self.device)
            result.append(self.lang.index2word[ni])
            count += 1
            if count == limit or (type(limit) == str and ni == self.lang.word2index[limit]) or count == 50 :
                stop = True
        if not retain :
            print(' '.join(result + ['\033[0m']))
            return
        else :
            return result, hidden   

In [184]:
class LanguageModelTrainer(object):
    def __init__(self, 
                 device,
                 criterion = nn.NLLLoss(reduce = False), #nn.BCEWithLogitsLoss(), #nn.BCELoss(), 
                 optimizer = optim.SGD,
                 print_every=100):
        # relevant quantities
        self.device = device
        self.criterion = criterion
        self.optimizer = optimizer
        self.print_every = print_every
        
        
    def asMinutes(self, s):
        m = math.floor(s / 60)
        s -= m * 60
        return '%dm %ds' % (m, s)
    
    
    def timeSince(self, since, percent):
        now = time.time()
        s = now - since
        es = s / (percent)
        rs = es - s
        return '%s (- %s)' % (self.asMinutes(s), self.asMinutes(rs))
        
        
    def trainLoop(self, agent, sentence, word, optimizer, learning_rate):
        """Performs a training loop, with forward pass and backward pass for gradient optimisation."""
        optimizer.zero_grad()
        agent.zero_grad()
        word_var     = agent.variableFromSentence([word]).view(-1)
        sentence_var = agent.variableFromSentence(sentence)
        log_probs, _ = agent.generateWord(sentence_var)
        
        loss = self.criterion(log_probs, word_var)
        topv, topi = log_probs[0].data.topk(1)
        loss_diff = 1 if topi.item() != word_var.data.item() else 0
        loss.backward()
        optimizer.step()                                
        return loss.data[0], loss_diff
        
        
    def train(self,
              agent, 
              sentences,
              n_iters = 100,
              learning_rate=0.01,
              random_state = 42
             ):
        """Performs training over a given dataset and along a specified amount of loops."""
        np.random.seed(random_state)
        start = time.time()
        optimizer = self.optimizer([param for param in agent.parameters() if param.requires_grad == True], lr=learning_rate)
        print_loss_total = 0  
        print_loss_diff_mots_total = 0
        for iter in range(1, n_iters + 1):
            sentence = random.choice(sentences) #--- [str]
            i = random.choice(range(len(sentence)))
            context = ['SOS'] + sentence[:i] 
            target = sentence[i]

            loss, loss_diff_mots = self.trainLoop(agent, context, target, optimizer, learning_rate)
            
            # affichage
            print_loss_total += loss
            print_loss_diff_mots_total += loss_diff_mots       
            if iter % (self.print_every) == 0:
                print_loss_avg = print_loss_total / self.print_every
                print_loss_diff_mots_avg = print_loss_diff_mots_total / self.print_every
                print_loss_total = 0
                print_loss_diff_mots_total = 0
                print('%s (%d %d%%) %.4f %.2f' % (self.timeSince(start, iter / n_iters),
                                             iter, iter / n_iters * 100, 
                                                  print_loss_avg, print_loss_diff_mots_avg))

### 1.2 Entrainement du modèle

In [212]:
def flatten(description) :
    '''Baisse le nombre de niveaux de 1 dans la description'''
    flatten = []
    for line in description :
        flatten += line
    return flatten
    

language_model_data = [flatten(text) for text in X]

In [561]:
# skipgram custom
#T = skipgram.embedding.weight.cpu().detach().numpy()
#lang = generateLanguageFromNumpy(X, init = 3, lvl = 2)

# fastText gensim
T = fastText_gensim.wv.vectors
lang = generateLanguageFromWordList(list(fastText_gensim.wv.vocab.keys()), init = 0)


language_model = LanguageModel(device = device,
                               lang = lang,
                               hidden_dim = 150, #100
                               embedding_weights = T,
                               n_layers = 3, #2
                               dropout = 0.15)
language_model = language_model.to(device)
language_model.nbParametres()

Mots comptés :  6473


1362623

In [266]:
language_model

LanguageModel(
  (embedding): Embedding(6473, 100)
  (dropout): Dropout(p=0.15)
  (gru): GRU(100, 100, num_layers=2, dropout=0.15)
  (out): Linear(in_features=100, out_features=6473, bias=True)
)

In [575]:
lang.addDescriptions(X)

freqs = list(lang.word2count.values())
freqs = [1/np.log(el+1) for el in freqs]
freqs = torch.tensor(freqs).to(device)

In [576]:
print(min(freqs), max(freqs))

tensor(0.0903, device='cuda:0') tensor(1.4427, device='cuda:0')


In [577]:
language_model_trainer = LanguageModelTrainer(device = device, 
                                              criterion = nn.NLLLoss(weight = freqs), 
                                              print_every = 500
                                             )

In [392]:
language_model.load_state_dict(torch.load(path_to_rep + '\saves\\demonstrateur_modele_linguistique.pth'))

In [595]:
language_model.train()
#language_model_trainer.train(language_model, language_model_data, n_iters = 40000, learning_rate=0.01)
#language_model_trainer.train(language_model, language_model_data, n_iters = 40000, learning_rate=0.005)
#language_model_trainer.train(language_model, language_model_data, n_iters = 40000, learning_rate=0.002)
language_model_trainer.train(language_model, language_model_data, n_iters = 40000, learning_rate=0.001)
#language_model_trainer.train(language_model, language_model_data, n_iters = 30000, learning_rate=0.0005)

0m 12s (- 17m 3s) (500 1%) 2.5293 0.42
0m 25s (- 16m 29s) (1000 2%) 2.5450 0.41
0m 37s (- 16m 7s) (1500 3%) 2.5891 0.41
0m 50s (- 16m 0s) (2000 5%) 2.2461 0.37
1m 2s (- 15m 42s) (2500 6%) 2.5721 0.42
1m 15s (- 15m 25s) (3000 7%) 2.7686 0.46
1m 26s (- 15m 0s) (3500 8%) 2.6629 0.43
1m 38s (- 14m 43s) (4000 10%) 2.6649 0.41
1m 49s (- 14m 27s) (4500 11%) 2.6879 0.43
2m 1s (- 14m 12s) (5000 12%) 2.8130 0.42
2m 14s (- 14m 2s) (5500 13%) 2.6266 0.43
2m 26s (- 13m 50s) (6000 15%) 2.2948 0.39
2m 39s (- 13m 43s) (6500 16%) 2.6697 0.42
2m 52s (- 13m 35s) (7000 17%) 2.5634 0.39
3m 6s (- 13m 29s) (7500 18%) 2.3739 0.40
3m 18s (- 13m 13s) (8000 20%) 2.8764 0.46
3m 30s (- 12m 59s) (8500 21%) 2.4915 0.41
3m 42s (- 12m 46s) (9000 22%) 2.4700 0.43
3m 56s (- 12m 37s) (9500 23%) 2.4957 0.43
4m 8s (- 12m 26s) (10000 25%) 2.6503 0.42
4m 22s (- 12m 16s) (10500 26%) 2.5782 0.43
4m 34s (- 12m 2s) (11000 27%) 2.4855 0.43
4m 46s (- 11m 49s) (11500 28%) 2.6465 0.41
4m 58s (- 11m 35s) (12000 30%) 2.5825 0.41
5m 9s

In [331]:
#torch.save(language_model.state_dict(), path_to_rep + '\saves\\Lilly_pkg_modele_linguistique.pth')

**Démo**

In [602]:
# fastText gensim, n_layers = 3, dh = 150
language_model.eval()
sentence = random.choice(language_model_data)
i = random.choice(range(len(sentence)))
sentence = sentence[:i] if i > 0 else ['SOS']
language_model(sentence, limit = 'EOS', color_code = '\x1b[48;2;255;229;217m' ) #  '\x1b[48;2;255;229;217m' '\x1b[31m'

short description : ast pmx pkg relance afficheur apollo yfspv ord . description : appel signaler afficheur yfspv ord affiche lot cours . relancer preparer visite zone executif demain matin . resolve notes : resume afficheur yfspv ord affiche lot cours . relancer preparer visite zone executif demain matin .  consequence factuelle impossible utiliser production . resolution . verification session limos . cbok . qualityassessment absence impact sispq valider . qaimpact impact qualite selon mail mar . resolution . ticket . qualityassessment absence impact sispq valider . qaimpact impact qualite selon mail mar . resolution . ticket . qualityassessment absence 
